In [2]:
import os

# Pushbullet api key
API_KEY = os.getenv("PUSHBULLET_API")
pb = Pushbullet(API_KEY)

# RSI calculator
def calculate_rsi(df, window=4):
    # Calculate daily price changes
    delta = df["Close"].diff()

    # Separate gains and losses
    gain = (delta.where(delta > 0, 0))
    loss = (-delta.where(delta < 0, 0))

    # Calculate the rolling average of gains and losses
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Compute the relative strength (RS)
    rs = avg_gain / avg_loss

    # Compute RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Add RSI to the DataFrame
    df["RSI"] = rsi
    df = df.dropna(subset=["RSI"])
    return df

# Decision matrix for the models
def decision_matrix(A, B, C):
    if A == 0:
        if B == 1 and C == 0:
            return 'SELL ETH'
        return 'SELL BTC'
    elif B == 0 and C == 1:
        return 'BUY ETH'
    else:
        return 'BUY BTC'

# Functions dropping an unnecessary row of data:
def dropping(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    return df

# Retrieves returns for data frames
def returns(df):
    df["Returns"] = (df["Close"] / df["Close"].shift(1)) - 1
    
    return df

# Printing important info (form of logging)
def printer(counter, r_states1, r_states2, r_states3):
    if counter == 19:
        print("Alive and kicking!")
        print("Last 10 HMM states:")
        print(r_states1.to_string() + "\n" + r_states2.to_string() + "\n" + r_states3.to_string())
        counter = 0
    elif counter == 0:
        print("Last 10 HMM states:")
        print(r_states1.to_string() + "\n" + r_states2.to_string() + "\n" + r_states3.to_string())
        counter += 1
    else:
        counter += 1

    return counter

def cts(h_states, s_counts, which):
    if which == 0:
        if s_counts.get(1, 0) > s_counts.get(0, 0):
            return [1 if state == 0 else 0 for state in h_states]
    else:
        if s_counts.get(1, 0) < s_counts.get(0, 0):
            return [1 if state == 0 else 0 for state in h_states]
            
    return h_states

def time_now():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def check_last_5_days(data):
    # Ensure the DataFrame is not empty
    if data.empty:
        return False

    # Ensure the index is a DateTimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("Data index is not a DateTimeIndex. Ensure correct data formatting.")

    # Get the last 5 days
    last_5_days = data.index[-5:]

    # Check if there are at least 5 entries
    if len(last_5_days) < 5:
        return False

    # Check if the last 5 dates are consecutive
    for i in range(1, len(last_5_days)):
        if (last_5_days[i] - last_5_days[i - 1]).days != 1:
            return False

    # Check if the last day matches today's date
    if last_5_days[-1].date() != datetime.now().date():
        return False

    return True

def HMM(df, components=2, covariance="full", iterations=100):
    X_train = df[["Returns", "RSI"]]
    model = GaussianHMM(n_components=components, covariance_type=covariance, n_iter=iterations).fit(X_train)
    
    return model.predict(X_train.values)

def state_sorting(hidden_states, which):
    state_counts = pd.Series(hidden_states).value_counts()
    if which == 0:
        if state_counts.get(1, 0) > state_counts.get(0, 0):
            hidden_states = [1 if state == 0 else 0 for state in hidden_states]
    else:
        if state_counts.get(1, 0) < state_counts.get(0, 0):
            hidden_states = [1 if state == 0 else 0 for state in hidden_states]

    return hidden_states

def printing_change_of_state(df, last_state, flag, recent_states, today_state):
    if today_state != last_state and flag != 1:
        message = f"State change detected at {time_now()}!\n\n"
        message += f"Today: {today_state}\nThe previous state: {last_state}\n"
        message += f"Latest HMM States:\n{recent_states.to_string()}"
        pb.push_note("HMM State Change Detected (runner)", message)
        last_state = today_state

    return last_state

def counter_printing(counter, recent_states):
    if counter >= 19:
        print("Alive and kicking!")
        print("Last 10 HMM states:")
        print(recent_states.to_string())
        print(time_now())
        counter = 0
    else:
        counter += 1

    return counter

def first_iteration(flag):
    if flag == 1:
        pb.push_note("Starting with states:", recent_states.to_string() + '\n' + time_now())
        last_state = today_state
        flag = 0

    return flag

SyntaxError: incomplete input (1851751095.py, line 138)